# Fermi-Hubbard Hamiltonian
___
This section creates the Fermi-Hubbard Hamiltonian with OpenFermion

In [1]:
# from scipy import sparse
from scipy.sparse import linalg
from openfermion import fermi_hubbard, jordan_wigner, get_sparse_operator
from openfermion.utils import count_qubits

In [374]:
"""Define the Hamiltonian."""
# Parameters from the paper

nsites = 4
occupation_number = 2
U = 10.0

filled_sites = int(nsites/2) # Half filling

# Hopping Integral

# In the paper U/t = 4
t = U / 4

# Is the sign of the tunneling right?
fermionic_hamiltonian = fermi_hubbard(1, nsites, tunneling=-t, coulomb=U, periodic=False)

# Map to QubitOperator using the JW transform
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

# Convert to Scipy sparse matrix
hamiltonian_jw_sparse = get_sparse_operator(jw_hamiltonian)

# Compute ground energy
eigs, _ = linalg.eigsh(hamiltonian_jw_sparse, k=1, which="SA")
ground_energy = eigs[0]

print(f"Ground State Energy: {ground_energy}")
print("JWT transformed Hamiltonian:")
print(jw_hamiltonian)

jw_hamiltonian_qubits = count_qubits(jw_hamiltonian)
print(jw_hamiltonian_qubits)

Ground State Energy: -6.562355678777142
JWT transformed Hamiltonian:
(10+0j) [] +
(1.25+0j) [X0 Z1 X2] +
(1.25+0j) [Y0 Z1 Y2] +
(-2.5+0j) [Z0] +
(2.5+0j) [Z0 Z1] +
(1.25+0j) [X1 Z2 X3] +
(1.25+0j) [Y1 Z2 Y3] +
(-2.5+0j) [Z1] +
(1.25+0j) [X2 Z3 X4] +
(1.25+0j) [Y2 Z3 Y4] +
(-2.5+0j) [Z2] +
(2.5+0j) [Z2 Z3] +
(1.25+0j) [X3 Z4 X5] +
(1.25+0j) [Y3 Z4 Y5] +
(-2.5+0j) [Z3] +
(1.25+0j) [X4 Z5 X6] +
(1.25+0j) [Y4 Z5 Y6] +
(-2.5+0j) [Z4] +
(2.5+0j) [Z4 Z5] +
(1.25+0j) [X5 Z6 X7] +
(1.25+0j) [Y5 Z6 Y7] +
(-2.5+0j) [Z5] +
(-2.5+0j) [Z6] +
(2.5+0j) [Z6 Z7] +
(-2.5+0j) [Z7]
8


# Pennylane Simulation

In [3]:
import pennylane as qml
from pennylane import numpy as np

In [375]:
# Translating the openfermion hamiltonian into a Pennylane qubit operator

def get_pennylane_hamiltonian(qubitoperator):

    pennylane_hamiltonian = qml.import_operator(qubitoperator)

    n_hamiltonian_qubits = len(pennylane_hamiltonian.wires)

    return pennylane_hamiltonian, n_hamiltonian_qubits

    # print(pennylane_hamiltonian)

    # print(qml.eigvals(pennylane_hamiltonian))

pennylane_hamiltonian, psi_qubits = get_pennylane_hamiltonian(jw_hamiltonian)


In [376]:
def add_cnots(cnot_pairs):

    for pair in cnot_pairs:

        control, target = pair

        qml.CNOT([control, target])

def change_to_z_basis(pauli_string):

    for pauli_tuple in pauli_string:

        qubit = pauli_tuple[0]
        pauli = pauli_tuple[1]
        if pauli == 'X':
            qml.Hadamard(qubit)
        if pauli == 'Y':
            qml.RX(-np.pi/2, qubit)

def pauli_time_evolution(pauli_string, coefficient, time, control = -1):

    if len(pauli_string) == 0:
        return None
    
    # Getting the CNOT pairs
    target_qubit = pauli_string[-1][0]

    cnot_pairs = []

    for pauli_tuple in pauli_string:

        control_qubit = pauli_tuple[0]

        if control_qubit != target_qubit:
            cnot_pairs.append([control_qubit, target_qubit])


    ###### Applying the circuit

    # Changing basis
    change_to_z_basis(pauli_string)

    # qml.Barrier(only_visual=True)

    # Adding CNOT gates
    if len(cnot_pairs) > 0:
        add_cnots(cnot_pairs)

    # qml.Barrier(only_visual=True)

    # Adding the rotation gate
    theta = 2 * coefficient * time 

    if control != -1:
        qml.ctrl(qml.RZ, control)(theta, target_qubit)
    else:
        qml.RZ(theta, target_qubit)

    # qml.Barrier(only_visual=True)

    # Undoing CNOT gates
    if len(cnot_pairs) > 0:
        cnot_pairs.reverse()
        add_cnots(cnot_pairs)

    # qml.Barrier(only_visual=True)

    # Undoing change of basis
    change_to_z_basis(pauli_string)

    # qml.Barrier(only_visual=True)


    

In [377]:

def first_order_trotter_decomposition(qubit_hamiltonian, time, trotter_number, control = -1):

    dt = time/trotter_number

    for i in range(trotter_number): 

        for pauli_string, coefficient in qubit_hamiltonian.terms.items():

            pauli_time_evolution(pauli_string, coefficient, dt, control = control)



In [378]:
def prepare_hartree_fock_state():

    for i in range(occupation_number):
        qml.PauliX(i)
    



In [379]:
n_ancilla_qubits = 1

n_circuit_qubits = jw_hamiltonian_qubits + n_ancilla_qubits

@qml.qnode(qml.device('default.qubit', wires = n_circuit_qubits, shots = 1))
def circuit(time, trotter_number, imaginary):


    ancilla_wire = n_circuit_qubits - 1

    qml.Barrier(only_visual=True)

    prepare_hartree_fock_state()

    qml.Barrier(only_visual=True)

    qml.Hadamard(ancilla_wire)

    qml.Barrier(only_visual=True)

    first_order_trotter_decomposition(jw_hamiltonian, time, trotter_number, control=ancilla_wire)

    qml.Barrier(only_visual=True)

    if imaginary:
        qml.adjoint(qml.S(ancilla_wire))

    qml.Hadamard(ancilla_wire)

    return qml.sample(wires=ancilla_wire)



# print(qml.draw(pauli_exponential)(((0, 'X'), (2, 'Z'), (3, 'Y')), 1.25j, 2.3))

print(qml.draw(circuit)(1, 1, True))


0: ──||──X──||─────||──RX(-1.57)─╭●───────────────────────╭●──RX(-1.57)──H─╭●───────────────────
1: ──||──X──||─────||────────────│──╭●─────────────────╭●─│────────────────│──╭●────────────────
2: ──||─────||─────||──RX(-1.57)─╰X─╰X─╭RZ(2.50+0.00j)─╰X─╰X──RX(-1.57)──H─╰X─╰X─╭RZ(2.50+0.00j)
3: ──||─────||─────||──────────────────│─────────────────────────────────────────│──────────────
4: ──||─────||─────||──────────────────│─────────────────────────────────────────│──────────────
5: ──||─────||─────||──────────────────│─────────────────────────────────────────│──────────────
6: ──||─────||─────||──────────────────│─────────────────────────────────────────│──────────────
7: ──||─────||─────||──────────────────│─────────────────────────────────────────│──────────────
8: ──||─────||──H──||──────────────────╰●────────────────────────────────────────╰●─────────────

─────────────╭●──H─────────────────────────────────────────────────────────────────────────────────
──╭●─────────│───RX(-1.57)

In [372]:
circuit(1, 50, False)

array(1)

In [334]:
def generate_X(j, tau):

    trotter_number = 1

    total_time = j * tau

    circuit_result = circuit(total_time, trotter_number, False)

    if circuit_result == 1: 
        return -1
    if circuit_result == 0:
        return 1

generate_X(1,1)

-1

In [356]:
def generate_Y(j, tau):

    trotter_number = 1

    total_time = j * tau

    circuit_result = circuit(total_time, trotter_number, True)

    if circuit_result == 1: 
        return -1
    if circuit_result == 0:
        return 1

generate_Y(1,1)

-1

In [ ]:
def generate_Z():
    

# Qulacs Simulation

In [7]:
"""Convert OpenFermion Hamiltonian to Qulacs Observable"""

from qulacs.observable import create_observable_from_openfermion_text
# from openfermion.utils import count_qubits

qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

n_qubits = qulacs_hamiltonian.get_qubit_count()


print(f"Succesfully created Qulacs Hamiltonian from OpenFermion Hamiltonian on {n_qubits} qubits")



Succesfully created Qulacs Hamiltonian from OpenFermion Hamiltonian on 8 qubits


In [83]:
from qulacs import QuantumState

state = QuantumState(n_qubits)
# state.set_Haar_random_state()

# setting up HF state
state.set_computational_basis(0b11000000)

qulacs_hamiltonian.get_expectation_value(state)

10.0

In [97]:
from qulacs import QuantumCircuit

circuit = QuantumCircuit(n_qubits+1)

# What is the angle?
angle = 0.1

# time steps
t_slice = 1

circuit.add_observable_rotation_gate(qulacs_hamiltonian, angle, t_slice)

print(circuit)

*** Quantum Circuit Info ***
# of qubit: 9
# of step : 21
# of gate : 25
# of 1 qubit gate: 8
# of 2 qubit gate: 4
# of 3 qubit gate: 12
Clifford  : no
Gaussian  : no


